In [47]:
#standard imports from your lecture
#a lot of these didn't work so I had to do the nltk.downloads
import os
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import download
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from string import punctuation
from datetime import datetime
import json
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from re import subn


In [48]:
#the above didn't work on COLAB so I had to do it like this
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
#Im always amazed how missing data can really screw up any manipulation of it
jeopardy = pd.read_json("jeopardy.json")
jeo = jeopardy.dropna()
question = list(jeo['question'])
values = list(jeo['value'])

In [50]:
english_stopwords = (stopwords.words('english') + list(punctuation))
token_list = [x for x in question if not x in english_stopwords]

In [51]:
#stemming of the questions
stemmer = PorterStemmer()
stemmed_qlist = []
stem_q = []
for x in token_list:
  stemmed_qlist.append(stemmer.stem(str(x)))

In [52]:
final_dict = pd.DataFrame([values, stemmed_qlist])
df = final_dict.T
df.columns = ['money_value','question']

In [53]:
#we need to sort the values into high and low, some of the values are $2000 and some are $2,000 so I had to take out commas
df.loc[:,'hi_low'] = df.loc[:,'money_value'].apply(lambda x: "low" if int(x[1:].replace(',','')) < 1000 else "high")


In [54]:
#taken from the last lecture video
X_train, X_test, y_train, y_test = train_test_split(df.question, 
                                                    df.money_value, random_state=1)
Tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_tf = Tfidf_vectorizer.fit_transform(X_train)
X_test_tf = Tfidf_vectorizer.transform(X_test)

In [55]:
#taken from the last lecture video
#its really innacurate but im proud of it
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_tf, y_train)
predictions = naive_bayes.predict(X_test_tf)
print("accuracy: ",accuracy_score(y_test,predictions))

accuracy:  0.19552171630035256
